In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
import undetected_chromedriver as uc
import time
from fractions import Fraction
from collections import defaultdict
from unicodedata import normalize

This code scrapes several betting odds from Oddschecker.com, converts the odds to percentages and calculates predicted points for players in the next full gameweek in Fantasy Premier League according to the percentages. In addition to selenium, webdriver has to be installed also. Webdrivers run or drive a browser from inside of your code. Version of webdriver has to match the version of your browser.

Assisting and Goalscoring odds for players are usually available couple of days before the game, so this script is very likely to return empty file if there are still several days until the first game of the gameweek.

In [ ]:
url = "https://fantasy.premierleague.com/api/fixtures/"
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Failed to fetch fixtures: {response.status_code}")
fixtures = response.json()

In [ ]:
game_weeks = defaultdict(list)
for fixture in fixtures:
    game_weeks[fixture["event"]].append(fixture)
for event in sorted(game_weeks.keys()):
    if all(not fixture['finished_provisional'] for fixture in game_weeks[event]):
        next_gameweek = event
        break
    else:
        next_gameweek = None

In [ ]:
url = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f"Failed to fetch teams: {response.status_code}")
data = response.json()
teams = data['teams']

In [ ]:
next_gw_fixtures = [fixture for fixture in fixtures if fixture['event'] == next_gameweek]

In [ ]:
TEAM_NAMES_ODDSCHECKER = {
    "Nott'm Forest": "Nottingham Forest",
    "Wolves": "Wolverhampton",
    "Spurs": "Tottenham",
    }

In [ ]:
# Function to normalize and prepare names for comparison
def prepare_name(name):
    """
    Normalizes a name by converting to lowercase, removing accents, and splitting into tokens.
    """

    # Replace Scandinavian letters with their ASCII equivalents
    scandinavian_replacements = {
        'ø': 'o',
        'å': 'a',
        'æ': 'ae',
        'Ø': 'O',
        'Å': 'A',
        'Æ': 'AE',
    }
    for scandinavian_char, ascii_char in scandinavian_replacements.items():
        name = name.replace(scandinavian_char, ascii_char)


    # Normalize the name to handle accents and foreign characters
    normalized_name = normalize('NFKD', name).encode('ascii', 'ignore').decode('ascii')
    # Convert to lowercase and split into tokens
    return normalized_name.lower().split()

In [ ]:
team_id_to_name = {team['id']: team['name'] for team in teams}
team_position = {team['id']: team['position'] for team in teams}
player_dict = {}
response = requests.get("https://fantasy.premierleague.com/api/bootstrap-static/")
player_data = response.json()
# Map element_type to position
element_types = {et["id"]: et["singular_name_short"] for et in data["element_types"]}

# Create a dictionary mapping player names to positions
player_positions = {}
for player in data["elements"]:
    player_name = player["first_name"] + " " + player["second_name"]  
    position_code = player["element_type"]
    player_positions[player_name] = element_types[position_code]


driver = uc.Chrome()  # Replace with the path to your WebDriver if needed
driver.get("https://www.oddschecker.com/football/english/premier-league/")
 
wait = WebDriverWait(driver, 10)

try:
    span_element = wait.until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[1]/div/section/h2/span[2]')))
    # Click on the <span> element (Accessung outside UK pop-up)
    span_element.click()

except TimeoutException:
    print()
    
wait = WebDriverWait(driver, 10)
try:
    cookiebutton = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'CookieBannerAcceptButton_c1mxe743')))
    # Click on the accept cookies button
    cookiebutton.click()
except TimeoutException:
    print()

except ElementClickInterceptedException:
    try:
        wait = WebDriverWait(driver, 10)
        cookiebutton = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'CookieBannerAcceptButton_c1mxe743')))
        cookiebutton.click()
    except ElementClickInterceptedException:
        print()
        
    
for fixture in next_gw_fixtures:
    home_team_id = fixture['team_h']
    away_team_id = fixture['team_a']
    home_team_name = team_id_to_name.get(home_team_id, "Unknown Team")
    away_team_name = team_id_to_name.get(away_team_id, "Unknown Team")

    wait = WebDriverWait(driver, 10)
    try:
        close_ad = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'webpush-swal2-close')))
        # Click close ad button
        close_ad.click()
    except TimeoutException:
        print()
        
    home_team = TEAM_NAMES_ODDSCHECKER.get(home_team_name, home_team_name)
    away_team = TEAM_NAMES_ODDSCHECKER.get(away_team_name, away_team_name)
    match_title = home_team + " v " + away_team
    
    matches_button = driver.find_element(By.XPATH, "//button[contains(text(), 'Matches')]")
    matches_button.click()
    
    # Find match link
    match_link = driver.find_element(By.XPATH, f"//a[@title='{match_title}'][@href]")
    href = match_link.get_attribute("href")
    driver.get(href)
        
    try:
        # Find the section header
        section_header = driver.find_element(By.XPATH, f"//h2[contains(text(), 'Player Assists')]")
        
        # Expand the section if it's collapsed
        if section_header.get_attribute("aria-expanded") == "false":
            section_header.click()
            time.sleep(3)  # Wait for the section to expand

        # Find all player names and odds in the section
        players = driver.find_elements(By.XPATH, f"//h2[contains(text(), 'Player Assists')]/following-sibling::*[1]/*[1]/*[1]//p")
        for player in players:

            try:
                name = player.get_attribute("innerText").strip(",Over 0.5")
        
                
                # Get the odds element (button following the player name)
                odd_element = player.find_element(By.XPATH, "./following-sibling::button")
                odd = odd_element.get_attribute("innerText").strip()

                # Initialize the player's dictionary if not already present
                if name not in player_dict:
                    player_dict[name] = {}

                # Add the odds to the player's dictionary
                player_dict[name]['Assisting Odd'] = odd

                # Calculate and add the probability
                probability = 1/(float(Fraction(odd) + 1))
                if probability is not None:
                    player_dict[name]["Assisting Probability"] = probability

            except Exception as e:
                print(f"Could not fetch odds for a player in 'Player Assists': {e}")

    except Exception as e:
        print("Could not find or expand section: Player Assists")
        
    try:
        # Find the section header
        section_header = driver.find_element(By.XPATH, f"//h2[contains(text(), 'To Score A Hat-Trick')]")
        
        # Expand the section if it's collapsed
        if section_header.get_attribute("aria-expanded") == "false":
            section_header.click()
            time.sleep(3)  # Wait for the section to expand

        # Find all player names and odds in the section
        players = driver.find_elements(By.XPATH, f"//h2[contains(text(), 'To Score A Hat-Trick')]/following-sibling::*[1]/*[1]/*[1]//p")
        for player in players:

            try:
                name = player.get_attribute("innerText").strip()
        
                
                # Get the odds element (button following the player name)
                odd_element = player.find_element(By.XPATH, "./following-sibling::button")
                odd = odd_element.get_attribute("innerText").strip()

                # Initialize the player's dictionary if not already present
                if name not in player_dict:
                    player_dict[name] = {}

                # Add the odds to the player's dictionary
                player_dict[name]['Scoring A Hat-Trick Odd'] = odd

                # Calculate and add the probability
                probability = 1/(float(Fraction(odd) + 1))
                if probability is not None:
                    player_dict[name]["Scoring A Hat-Trick Probability"] = probability

            except Exception as e:
                print(f"Could not fetch odds for a player in 'To Score A Hat-Trick': {e}")

    except Exception as e:
        print("Could not find or expand section: To Score A Hat-Trick")
        
    try:
        # Find the section header
        section_header = driver.find_element(By.XPATH, f"//h2[contains(text(), 'To Score 2 Or More Goals')]")
        
        # Expand the section if it's collapsed
        if section_header.get_attribute("aria-expanded") == "false":
            section_header.click()
            time.sleep(3)  # Wait for the section to expand

        # Find all player names and odds in the section
        players = driver.find_elements(By.XPATH, f"//h2[contains(text(), 'To Score 2 Or More Goals')]/following-sibling::*[1]/*[1]/*[1]//p")
        for player in players:

            try:
                name = player.get_attribute("innerText").strip()
        
                
                # Get the odds element (button following the player name)
                odd_element = player.find_element(By.XPATH, "./following-sibling::button")
                odd = odd_element.get_attribute("innerText").strip()

                # Initialize the player's dictionary if not already present
                if name not in player_dict:
                    player_dict[name] = {}

                # Add the odds to the player's dictionary
                player_dict[name]['Scoring 2 Or More Goals Odd'] = odd

                # Calculate and add the probability
                probability = 1/(float(Fraction(odd) + 1))
                if probability is not None:
                    player_dict[name]["Scoring 2 Or More Goals Probability"] = probability

            except Exception as e:
                print(f"Could not fetch odds for a player in 'To Score 2 Or More Goals': {e}")

    except Exception as e:
        print("Could not find or expand section: To Score 2 Or More Goals")
        
    try:
        # Find the section header
        section_header = driver.find_element(By.XPATH, f"//h2[contains(text(), 'Anytime Goalscorer')]")
        
        # Expand the section if it's collapsed
        if section_header.get_attribute("aria-expanded") == "false":
            section_header.click()
            time.sleep(3)  # Wait for the section to expand

        # Find all player names and odds in the section
        players = driver.find_elements(By.XPATH, f"//h2[contains(text(), 'Anytime Goalscorer')]/following-sibling::*[1]/*[1]/*[1]//p")
        for player in players:

            try:
                name = player.get_attribute("innerText").strip()
        
                
                # Get the odds element (button following the player name)
                odd_element = player.find_element(By.XPATH, "./following-sibling::button")
                odd = odd_element.get_attribute("innerText").strip()

                # Initialize the player's dictionary if not already present
                if name not in player_dict:
                    player_dict[name] = {}

                # Add the odds to the player's dictionary
                player_dict[name]['Anytime Scoring Odd'] = odd

                # Calculate and add the probability
                probability = 1/(float(Fraction(odd) + 1))
                if probability is not None:
                    player_dict[name]["Scoring Anytime Probability"] = probability

            except Exception as e:
                print(f"Could not fetch odds for a player in 'Anytime Goalscorer': {e}")

    except Exception as e:
        print("Could not find or expand section: Anytime Goalscorer")
    
    # Add positions to player_dict
    for player in player_dict:
        # Prepare the player name for comparison
        player_tokens = prepare_name(player)

        # Check if the player name matches any key in player_positions
        matched_position = None
        for fpl_player, position in player_positions.items():
            # Prepare the FPL player name for comparison
            fpl_tokens = prepare_name(fpl_player)

            # Check if all tokens in one name exist in the other
            if all(token in fpl_tokens for token in player_tokens) or all(token in player_tokens for token in fpl_tokens):
                matched_position = position
                break  # Stop searching once a match is found

        # Assign the position (or "Unknown" if no match is found)
        player_dict[player]["Position"] = matched_position if matched_position else "Unknown"
        
    """
    Calculates the "Points" column for each player based on the provided formula.
    """
    for player, odds in player_dict.items():
        try:
            # Get probabilities
            anytime_prob = odds.get("Anytime Goalscoring Odd Probability", 0)
            two_or_more_prob = odds.get("Scoring 2 Or More Goals Odd Probability", 0)
            hattrick_prob = odds.get("Scoring A Hat-Trick Odd Probability", 0)
            assisting_prob = odds.get("Assisting Odd Probability", 0)

            # Calculate points
            if player_dict[player]['Position'] == 'MID':
                points = (
                3 * hattrick_prob * 5 +
                2 * (two_or_more_prob - hattrick_prob) * 5 +
                (anytime_prob - two_or_more_prob) * 5 + assisting_prob * 3
                )
            if player_dict[player]['Position'] == 'DEF':
                points = (
                3 * hattrick_prob * 6 +
                2 * (two_or_more_prob - hattrick_prob) * 6 +
                (anytime_prob - two_or_more_prob) * 6 + assisting_prob * 3
                )
            else:
                points = (
                3 * hattrick_prob * 4 +
                2 * (two_or_more_prob - hattrick_prob) * 4 +
                (anytime_prob - two_or_more_prob) * 4 + assisting_prob * 3
                )
            player_dict[player]["Points"] = round(points, 3)  # Round to 3 decimal places
        except Exception as e:
            print(f"Could not calculate points for {player}: {e}")
  
    driver.get("https://www.oddschecker.com/football/english/premier-league/")
    
    

driver.quit()

In [ ]:
player_data_df = pd.DataFrame.from_dict(player_dict, orient='index')
player_data_df.index.name = 'Player'
print(player_data_df.head())
player_data_df.to_excel("players_output.xlsx")